# Analysing Genuine files in Eval set - version 2.0 dataset

In [16]:
%%bash 
ls /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_train/ | wc -l

0


ls: cannot access '/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_train/': No such file or directory


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [18]:
# Load the prediction file
# Predictions file that has three columns: genuin, spoofed prob and log likehood ratio

pred_file='model_3sec_relu_0.5_run8/predictions/eval_prediction.txt'
#new_pred_file = 'model_3sec_relu_0.5_run8/predictions/eval_prediction_with_index.txt'
new_pred_file = 'predictions/eval_prediction_with_index.txt'

In [19]:
# Create a new prediction file where we write index (starting from 0) as the first column
# This way things become much easier

'''
with open(pred_file) as f, open(new_pred_file,'w') as f2:
    i=0
    for line in f:
        f2.write(str(i)+' '+ line)
        #print(i)
        i+=1    
'''

# Above code is just used to append file index in the prediction file
# Careful when re-running

"\nwith open(pred_file) as f, open(new_pred_file,'w') as f2:\n    i=0\n    for line in f:\n        f2.write(str(i)+' '+ line)\n        #print(i)\n        i+=1    \n"

In [20]:
%%bash

#cat model_3sec_relu_0.5_run8/predictions/eval_prediction_with_index.txt | tail
cat predictions/eval_prediction_with_index.txt | head


 #Check the new prediction file

0 0.999724 0.000276201 8.19411
1 0.9991 0.000899707 7.01254
2 0.999855 0.000145446 8.83556
3 0.999999 8.39425e-07 13.9905
4 0.999989 1.06374e-05 11.4511
5 0.998049 0.00195086 6.23753
6 0.882768 0.117232 2.01891
7 0.9999 0.000100108 9.20916
8 0.999961 3.94849e-05 10.1396
9 0.999989 1.11043e-05 11.4082


In [21]:
# Separate out predictions of genuine and spoofed files
# Eval set set has first 1298 as genuine and remaining 12008 are spoofed
# This is based on the protocal file I created which was used during training and feature extraction

all_predictions = []
all_gens = []
all_spoofs = []

with open(new_pred_file) as f:
    all_predictions = [line.strip() for line in f]
    
all_gen_predictions = all_predictions[:1298]                 
all_spf_predictions = all_predictions[1298:]                 

In [22]:
print(len(all_gen_predictions))
print(len(all_spf_predictions))

1298
12008


In [23]:
# Collect all genuine file for which genuine class got probability > 90%
# the strongly correctly classified cases

'''
fname='index_file_list/allGenIndexList_TP.txt'
count=0
splitIndex = 1     # 1 for genuine and 2 for spoofed (the third column)
predList = all_gen_predictions   # use all_spf_predictions for spoofed case

#print(len(predList))

with open(fname,'w') as f2:
    for items in predList:
        prob = items.strip().split(' ')[splitIndex]   
        if float(prob) > 0.9:
            #print(items)
            count+=1
            f2.write(items+'\n')            
print(count)
'''
# Careful when you re-run this code. It is use to take files with 90% probability to do analysis using slime !

"\nfname='index_file_list/allGenIndexList_TP.txt'\ncount=0\nsplitIndex = 1     # 1 for genuine and 2 for spoofed (the third column)\npredList = all_gen_predictions   # use all_spf_predictions for spoofed case\n\n#print(len(predList))\n\nwith open(fname,'w') as f2:\n    for items in predList:\n        prob = items.strip().split(' ')[splitIndex]   \n        if float(prob) > 0.9:\n            #print(items)\n            count+=1\n            f2.write(items+'\n')            \nprint(count)\n"

In [24]:
%%bash
cat index_file_list/allGenIndexList_TP.txt | wc -l

1205


## Total genuine files we got with > 90% probability is = 1205 (out of 1298)

> Take the top 10 genuine confident correctly classified files - high scores

                               Gen prob     Spf Prob      Log Liklihood ratio
    
       
**** Note when accessing via list or array, index 575 should be called as 574       

## Analysing Time: The top two components from SLIME - True positive Genuine case

Note, that under time analysis, we have cut our input spectrogram into 10 different temporal components/segments, where each segment correpsonds to

> ***300 mili seconds***

In [25]:
# Show how we cut the spectrogram in timexfrequency

Image("time.png")

In [26]:
# Load the temporal explanantion file produced by slime

file = 'top_two_explanation_indices/time/gen_TP_box.txt'    #using box for spectrogram computation

***Make sure to use the correct spectrogram for two cases !! ***

In [27]:
%%bash 

cat top_two_explanation_indices/time/gen_TP_box.txt | wc -l

1205


In [28]:
def get_top1_top2_list(file):
    with open(file) as f:
        top1 = [int(line.strip().split(' ')[0]) for line in f]
    with open(file) as f:  
        top2 = [int(line.strip().split(' ')[1]) for line in f]
    return top1, top2    

In [29]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [30]:
print(len(top1))
print(len(top2))

1205
1205


In [31]:
def get_counts(datalist, key):
    count=0
    for i in range(0,len(datalist)):
        if datalist[i] == key:
            count+=1
    return count            

In [32]:
def print_component_distribution(comps, predList, order):
    print('Printing component weigting distribution for Top:', order)
    for i in comps:
        print('Component ' + str(i) + ' : ' + str(get_counts(predList, i)))    

### Using the top1 components - given by SLIME (Time segmentation) - Genuine Class

   
    Component 0 : 39
    Component 1 : 1
    Component 2 : 172
    Component 3 : 539
    Component 4 : 52
    Component 5 : 254
    Component 6 : 126
    Component 7 : 20
    Component 8 : 0
    Component 9 : 2


In [37]:
39+1+172+539+52+254+126+20+2

1205

In [38]:
comps=[0,1,2,3,4,5,6,7,8,9]    # In time we have 10 segments/components

In [39]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 39
Component 1 : 1
Component 2 : 172
Component 3 : 539
Component 4 : 52
Component 5 : 254
Component 6 : 126
Component 7 : 20
Component 8 : 0
Component 9 : 2


From above we see that from 766 genuine TP audio files (that got scores > 0.5) we find that the largest number of times, SLIME gives highest weightage to Component 4 (271 times out of 766). Component 4 corresponds to 900-1200ms audio signal. This is then followed by Component 1 (136) which is the first 300 ms signal.

Using signal Box also we found the same behaviour.

 * What to do next?
We now pick those 271 audio files that got top1 as component 4 and try to find if we get some meaningful information 

# Pick 5 audio files for hearing

In [40]:
#random_5_ids = [506,480,757,208,481]   # Genuine True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [507,481,758,209,482]   # Genuine True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [41]:
%%bash
#ls /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000481.wav

In [42]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000482.wav audio_files_hearing/genuine_TP/

#play audio_files_hearing/genuine_TP/D_1000507.wav
#ls audio_files_hearing/genuine_TP/

## Analysing Freq: The top two components from SLIME - True positive Genuine case

Note, that under frequency analysis, we have cut our input spectrogram into 8 different frequency components/segments, where each segment correpsonds to

> ***1000 Hz frequency***

In [45]:
# Show how we cut the spectrogram in timexfrequency

Image("frequency.png")

In [46]:
# Load the spectral explanation file

file = 'top_two_explanation_indices/freq/gen_TP_box.txt'   #using signal box

In [47]:
%%bash
#cat 'top_two_explanation_indices/freq/gen_TP.txt' | head

# the top two components 7 6 dominates the explanation

In [48]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [49]:
print(len(top1))
print(len(top2))

1205
1205


## Top 1 component distribution - Frequency

In [50]:
comps = [0,1,2,3,4,5,6,7]   # in Frequency we have 8 components

In [51]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 1
Component 1 : 0
Component 2 : 0
Component 3 : 0
Component 4 : 0
Component 5 : 26
Component 6 : 0
Component 7 : 1178


In [52]:
1+26+1178

1205

# Pick 5 audio files for hearing

In [241]:
#random_5_ids = [571,455,387,202,708]   # Genuine True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [572,456,388,203,709]   # Genuine True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [106]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000709.wav audio_files_hearing/genuine_TP/

#play audio_files_hearing/genuine_TP/D_1000507.wav
#ls audio_files_hearing/genuine_TP/

# On Evaluation set

The evaluation set has 1298 genuine and 12008 spoofed examples. 



## What CNN has exploited about Genuine signal from the training data ?

We first collect all the genuine audio files that has been strongly correctly classified with more than 90% probability by the CNN model. We find 1205 audio files (out of 1298). Next, we run SLIME algorithm on these 1205 audio files and generate temporal and spectral explanation distribution.

Distribution of **temporal explanation** for 

    Component 0 : 39
    Component 1 : 1
    Component 2 : 172
    Component 3 : 539
    Component 4 : 52
    Component 5 : 254
    Component 6 : 126
    Component 7 : 20
    Component 8 : 0
    Component 9 : 2
    
    
Distribution of **Spectral explanation**

    Component 0 : 1
    Component 1 : 0
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 26
    Component 6 : 0
    Component 7 : 1178
    
    
   
On average, the explanation distribution show strong emphasis in the center of the 3 second audio signal. This further corresponds the region in audio signal where speech onset is detected. We find that majority of genuine audio files in the development data has non-speech frames in the beginning. On the spectral explanation distribution, it gives a very clear explanation that the model is looking at high frequency information above 7 kHz. 

> Therefore, what **conclusion** we get from above is that th model is identifying a file as genuine using information above 7 kHz mostly in the middle of the audio signal.
 






